# Sentence to PClean query demo
This notebook demos the sentence-to-PClean pipeline. Given a sentence giving information about some doctor, it generates and runs queries about those people against the Medicare dataset used in Alex Lew's paper using a PClean model trained by Ian Limarta.


In [55]:
import gc
import json
import logging
from pathlib import Path
import sqlite3
import string
from typing import Any, Optional, TypeVar, Union

from IPython.display import display, Markdown
import lark
import nest_asyncio
import requests
import spacy
import torch
import transformers
from tqdm import tqdm
import vllm

import genparse
from genparse import InferenceSetup, InferenceSetupVLLM

nest_asyncio.apply()

logger = logging.getLogger("notebook")

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
)

In [2]:
debug_sentences_path = Path().resolve() / "debug_sentences.jsonl"
debug_sentences = [json.loads(line) for line in debug_sentences_path.read_text(encoding="utf-8").splitlines() if line.strip()]

In [3]:
detailed_debug_sentences_path = Path().resolve() / "debug_sentences_detailed.jsonl"
detailed_debug_sentences = [json.loads(line) for line in detailed_debug_sentences_path.read_text(encoding="utf-8").splitlines() if line.strip()]

## Example inputs and outputs

Taken from the first ten sentences (rows 2-11) in the [GPT-4 tweets Marjorie Freedman generated][gpt4_tweets].

[gpt4_tweets]: https://docs.google.com/spreadsheets/d/1vq_HAdbFY079vYVOppkFLk_zojJ46MJKsy-TzUm7Q3M/edit

In [4]:
sentence_table_row_template = string.Template("| $sentence | <pre>$example_output</pre> |")
def show_sentences(sentence_data):
    lines = ["| Sentence | Example PClean preamble code |", "| --- | --- |"]
    for sentence_datum in sentence_data:
        lines.append(sentence_table_row_template.substitute(sentence=sentence_datum["sentence"], example_output=sentence_datum["example_output"].replace("\n", "<br>")))
    markdown = "\n".join(lines)
    display(Markdown(markdown))


show_sentences(debug_sentences)

| Sentence | Example PClean preamble code |
| --- | --- |
| Just had an insightful consultation with Dr. Kay Ryan in Baltimore. Feeling optimistic about my health! #HealthMatters #Baltimore | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Dr. Kay Ryan's office at 321 Pine St. in Baltimore is so welcoming and efficient. Highly recommend! #CityCare | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)<br>row_trace[address_key] = "321 Pine St."</pre> |
| Exploring Baltimore after an informative appointment with Dr. Kay Ryan. Loving this city's vibe! #BaltimoreAdventures #DoctorVisit | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Dr. Kay Ryan, a neurologist, provided excellent care today. Grateful for her expertise! #Healthcare #BaltimoreDoctors | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>occupation_key = PClean.resolve_dot_expression(trace.model, :Obs, :occupation)<br>row_trace[occupation_key] = "neurologist"</pre> |
| Feeling reassured after my visit to Dr. Kay Ryan in Baltimore. She's truly exceptional! #FeelingGood #CityOfCharm | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Had a great experience with Dr. Kay Ryan at 321 Pine St. in Baltimore. Her attention to detail is unmatched! #HealthCheck | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)<br>row_trace[address_key] = "321 Pine St."</pre> |
| From diagnosis to treatment, Dr. Kay Ryan, a cardiologist, covered it all. Baltimore, you're lucky to have her! #MedicalCare | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>occupation_key = PClean.resolve_dot_expression(trace.model, :Obs, :occupation)<br>row_trace[occupation_key] = "cardiologist"</pre> |
| Just wrapped up my appointment with Dr. Kay Ryan in Baltimore. Her professionalism is top-notch. #HealthJourney | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Dr. Kay Ryan's office in Baltimore is so efficient and welcoming. A great experience overall! #DoctorVisit #CityCare | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Can't say enough good things about Dr. Kay Ryan, a pediatrician, in Baltimore. Truly an outstanding doctor! #Grateful | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>occupation_key = PClean.resolve_dot_expression(trace.model, :Obs, :occupation)<br>row_trace[occupation_key] = "pediatrician"</pre> |

## Helper code

### SpaCy code

In [35]:
# Supporting code for entity extraction.
_SPACY_MODEL_NAME = 'en_core_web_trf'
spacy_model = spacy.load(_SPACY_MODEL_NAME)
_PERSON_LABEL = 'PERSON'
_LOCATION_LABEL = 'LOC'
# spaCy labels list example thanks to Stack Overflow user 'russhoppa': https://stackoverflow.com/a/78252807
_SPACY_LABELS = spacy_model.get_pipe("ner").labels
assert _PERSON_LABEL in _SPACY_LABELS

T = TypeVar('T')

def _uniquify(items: list[T]) -> list[T]:
    """
    O(n^2) order-preserving uniquification.

    Fine for short lists like a single sentence's list of PERSON entities.
    """
    result = []
    for item in items:
        if item not in result:
            result.append(item)
    return result

def get_people(sentence: str) -> list[str]:
    return _uniquify(
        [ent.text for ent in spacy_model(sentence).ents if ent.label_ == _PERSON_LABEL]
    )

def get_locations(sentence: str) -> list[str]:
    return _uniquify(
        [ent.text for ent in spacy_model(sentence).ents if ent.label_ == _LOCATION_LABEL]
    )

def show_ents(sentence: str) -> None:
    print(sentence)
    print(spacy_model(sentence).ents)

for sentence_datum in debug_sentences:
    show_ents(sentence_datum["sentence"])
    print()

Just had an insightful consultation with Dr. Kay Ryan in Baltimore. Feeling optimistic about my health! #HealthMatters #Baltimore
(Kay Ryan, Baltimore, Baltimore)

Dr. Kay Ryan's office at 321 Pine St. in Baltimore is so welcoming and efficient. Highly recommend! #CityCare
(Kay Ryan, Baltimore, CityCare)

Exploring Baltimore after an informative appointment with Dr. Kay Ryan. Loving this city's vibe! #BaltimoreAdventures #DoctorVisit
(Baltimore, Kay Ryan)

Dr. Kay Ryan, a neurologist, provided excellent care today. Grateful for her expertise! #Healthcare #BaltimoreDoctors
(Kay Ryan, today)

Feeling reassured after my visit to Dr. Kay Ryan in Baltimore. She's truly exceptional! #FeelingGood #CityOfCharm
(Kay Ryan, Baltimore)

Had a great experience with Dr. Kay Ryan at 321 Pine St. in Baltimore. Her attention to detail is unmatched! #HealthCheck
(Kay Ryan, Baltimore)

From diagnosis to treatment, Dr. Kay Ryan, a cardiologist, covered it all. Baltimore, you're lucky to have her! #Medical

### Extracting the generated code from a response

In [58]:
def extract_code_from_response(text: str) -> str:
    result = text.removeprefix('<|start_header_id|>assistant<|end_header_id|>').strip()
    return result

In [37]:
# Test to confirm it works as intended
text = '''<|start_header_id|>assistant<|end_header_id|>

Here is the Julia code to query the PClean table of records about doctors based on the given input sentence:

```julia
name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)
row_trace[name_key] = "Kay Ryan"
city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)
row_trace[city_key] = "Baltimore"
```'''
extract_code_from_response(text)

'name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)\nrow_trace[name_key] = "Kay Ryan"\ncity_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)\nrow_trace[city_key] = "Baltimore"'

### Sort posterior

In [38]:
def sort_posterior(posterior):
    return {inference: likelihood for inference, likelihood in sorted(posterior.items(), key=lambda t: (t[1], t[0]), reverse=True)}

### Aggregate likelihoods over extracted code

In [39]:
def get_aggregate_likelihoods(posterior: dict[str, float]) -> dict[str, float]:
    result = {}
    for inference, likelihood in posterior.items():
        code_only = extract_code_from_response(inference)
        result.setdefault(code_only, 0.0)
        result[code_only] += likelihood
    return sort_posterior(result)

### Get best inference

In [40]:
def get_best_inference(posterior: dict[str, float]) -> tuple[str, float]:
    return max(posterior.items(), key=lambda t: (t[1], t[0]))

### Running inference locally

In [71]:
SERVER_MODEL_ID = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
SERVER_GENPARSE_MODEL_NAME = "llama3.1"
_LOCAL_BATCH_SIZE = 1

class FakeResponse:
    def __init__(self, data):
        self._data = data
    
    @property
    def status_code(self):
        return 200

    def json(self):
        return self._data

    def text(self):
        return json.dumps(self._data)

def server_model(grammar, proposal="character"):
    return InferenceSetupVLLM(SERVER_GENPARSE_MODEL_NAME, grammar, proposal_name=proposal)

def run_inference_vllm(
    prompt: str,
    *,
    proposal: str = 'character',
    batch_size: int = _LOCAL_BATCH_SIZE,
    max_tokens: int,
    n_particles: int,
    temperature: float = 1.0,
    grammar: str,
    genparse_url: str = "",
    inference_setup: InferenceSetupVLLM,
) -> requests.Response:
    #assert batch_size == inference_setup.sampler.llm._model.batch_size
    assert grammar == inference_setup.sampler.grammar
    assert (proposal == 'character' and isinstance(inference_setup, genparse.CharacterProposal))
    inference = inference_setup(prompt, n_particles=n_particles, temperature=temperature, max_tokens=max_tokens)
    return FakeResponse({"log_ml": None, "posterior": inference.posterior})

### Running the server

In [53]:
_BATCH_SIZE = 1
# Ben LeBrun's WIP server running on GCP as of 2024-07-16
_DEFAULT_GENPARSE_INFERENCE_SERVER_URI = 'http://34.122.30.137:8888/infer'

def run_inference_server(
    prompt: str,
    *,
    proposal: str = 'character',
    batch_size: int = _BATCH_SIZE,
    max_tokens: int,
    n_particles: int,
    temperature: float = 1.0,
    grammar: str,
    genparse_url: str = _DEFAULT_GENPARSE_INFERENCE_SERVER_URI,
) -> requests.Response:
    """
    Run inference using a server.
    """
    params = {
        'prompt': prompt,
        'method': 'smc-standard',
        'n_particles': n_particles,
        'lark_grammar': grammar,
        'proposal_name': proposal,
        'proposal_args': {},
        'max_tokens': max_tokens,
        'temperature': temperature,
    }
    headers = {
        "Content-type": "application/json",
        "Accept": "application/json"
    }
    response = requests.post(genparse_url, json=params, headers=headers)

    return response

### PClean grammar

In [44]:
pclean_grammar = r"""
start: prefix julia_code suffix
prefix: "<|start_header_id|>assistant<|end_header_id|>" NL* FREE_TEXT? NL+ CODE_FENCE JULIA? NL
suffix: NL CODE_FENCE
julia_code: add_to_trace (NL+ add_to_trace)+

FREE_TEXT: /[a-zA-Z0-9.,-?!;: ]+/
CODE_FENCE: "```"
JULIA: "julia"
WS: " "
NL: "\n"
STRING: /"[a-zA-Z0-9. ]*"/

add_to_trace: get_key NL set_key_in_trace
# slightly overly restrictive but good enough 
get_key: trace_key_identifier WS* "=" WS* "PClean.resolve_dot_expression(trace.model, :Obs, " column_symbol ")"
# column_symbol: /:[a-z][a-z_]+/
column_symbol: ":" ("name" | "address" | "specialty" | "city")
set_key_in_trace: "row_trace[" trace_key_identifier "]" WS* "=" WS* STRING
trace_key_identifier: /[a-z][a-z_]+/
"""
pclean_parser = lark.Lark(pclean_grammar)

### PClean code template

In [45]:
pclean_template = string.Template(
    """
# Create a new row trace for the hypothetical row
row_trace = Dict{PClean.VertexID, Any}()
$preamble

# Add it to the trace
obs = trace.tables[:Obs].observations
row_id = gensym()
obs[row_id] = row_trace

samples = []
for _ in 1:$N
    # Perform a Partilce Gibbs MCMC move to change our current sample of the row
    PClean.run_smc(!(trace, :Obs, row_id, PClean.InferenceConfig(1, 10))
    # Accumulate the sample
    push!(samples, trace.tables[:Obs].rows[row_id][br_idx]
end

countmap(samples)
""".lstrip()
)

PCLEAN_DEFAULT_N_SAMPLES = 100

### PClean generation prompt

In [46]:
pclean_prompt = string.Template(
    """Write Julia code to query a PClean table of records about doctors based on the given input sentence.

In general, your output should look like pairs of lines:

```julia
blah_key = PClean.resolve_dot_expression(trace.model, :Obs, :blah_col)
row_trace[blah_key] = "Value of Blah as expressed in the sentence"
```

The dataset has just four columns to query:

- :name (the doctor's full name, first and last)
- :city (the city where the doctor practices)
- :address (the doctor's office address)
- :specialty (the doctor's specialty)

Please generate code to query all values specified in the sentence. Output the Julia code directly with no preamble or commentary. Write just two lines per column.

Input: Loved visiting Dr. Kay Ryan's neurology office at 256 Overflow St! No wait time at all. #Baltimore
Output: ```julia
name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)
row_trace[name_key] = "Kay Ryan"
address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)
row_trace[address_key] = "256 Overflow St"
specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)
row_trace[specialty_key] = "neurology"
city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)
row_trace[city_key] = "Baltimore"
```

Input: Dr. Pat Rogers's orthopedics office screwed us! Took our money and Kay gave us three minutes tops. #BaltimoreSucks
Output: ```julia
name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)
row_trace[name_key] = "Pat Rogers"
specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)
row_trace[specialty_key] = "orthopedics"
city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)
row_trace[city_key] = "Baltimore"
```

In other words, we query all values given in the sentence: The doctor's name, their office address, their specialty, and the city. If one of these is missing, we do not query on it.

Input: $sentence
Output:"""
)
# scrapped
# The doctor's name is $name. Their office address is $address. Their specialty is $specialty. The city is $city.

def format_people_prompt(sentence: str) -> str:
    return pclean_prompt.substitute(sentence=sentence)

### Running PClean

In [47]:
def run_pclean(code: str) -> Any:
    # TODO
    raise NotImplementedError()

### Displaying PClean query results

In [48]:
def display_query_results(query_results: Any) -> None:
    # TODO
    raise NotImplementedError()

### JSON generation prompt

In [49]:
json_prompt = string.Template(
    """Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's first name)
- "last" (the doctor's last name)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" -- this is the street address)
- "addr2" (additional address information giving a suite number, unit number, et cetera -- "line 2" of the address)

Please generate a key-value pair for every such value specified in the sentence. Avoid nulls and empty strings. Omit any missing values. Do not write a script. Output the JSON itself directly with no preamble or commentary. The following examples illustrate how you should behave on the input sentence.

Input: John Smith's neurology office (Happy Brain Services LLC) at 512 Example Street Suite 3600 (Camp Hill) is terrible!
Output: ```json
{"first": "John", "last": "Smith", "specialty": "neurology", "legal_name": "Happy Brain Services LLC", "addr": "512 Example Street", "addr2": "Suite 3600", "city_name": "Camp Hill"}
```

Input: Loved visiting Dr. Kay Ryan's neurology office (Green Medicine Inc.) at 256 Overflow St (ZIP 17011-2202)! No wait time at all. #CampHill
Output: ```json
{"first": "Kay", "last": "Ryan", "legal_name": "Green Medicine Inc", "specialty": "neurology", "addr": "256 Overflow St", "zip": "170112202", "city_name": "Camp Hill"}
```

Input: Dr. Pat Rogers's office screwed us! So much for Soulful Medical Services Inc. Took our money and Pat gave us three minutes tops. #BaltimoreSucks
Output: ```json
{"first": "Pat", "last": "Rogers", "legal_name": "Soulful Medical Services Inc", "city_name": "Baltimore"}
```

The following is your input sentence. Produce the appropriate output.

Input: $sentence
Output:"""
)
# scrapped
# The doctor's name is $name. Their office address is $address. Their specialty is $specialty. The city is $city.

def format_json_prompt(sentence: str) -> str:
    return json_prompt.substitute(sentence=sentence)

print(json_prompt)

### JSON grammar

In [50]:
json_grammar = r"""
start: prefix json suffix
prefix: "<|start_header_id|>assistant<|end_header_id|>" NL* FREE_TEXT? NL+ CODE_FENCE JSON_TAG? NL
suffix: NL CODE_FENCE
json: "{" NL? WS* kv_pair ("," NL? WS* kv_pair)* NL? WS* "}"
kv_pair: COLUMN WS* ":" WS* STRING

FREE_TEXT: /[a-zA-Z0-9.,-?!;:' ]+/
CODE_FENCE: "```"
JSON_TAG: "json"
WS: " "
NL: "\n"
STRING: /"[a-zA-Z0-9-.,'& ]*"/
# dropped for now: "city"
COLUMN: "\"" ("first_name" | "last_name" | "address" | "address2" | "specialty" | "c2z3" | "legal_name") "\""
"""
json_parser = lark.Lark(json_grammar)

In [51]:
force_nonempty_json_grammar = r"""
start: prefix json suffix
prefix: "<|start_header_id|>assistant<|end_header_id|>" NL* FREE_TEXT? NL+ CODE_FENCE JSON_TAG? NL
suffix: NL CODE_FENCE
json: "{" NL? WS* kv_pair ("," NL? WS* kv_pair)* NL? WS* "}"
#kv_pair: COLUMN WS* ":" WS* STRING
kv_pair: (NONEMPTY_COLUMN WS* ":" WS* NONEMPTY_STRING) | (EMPTYABLE_COLUMN WS* ":" WS* STRING)

FREE_TEXT: /[a-zA-Z0-9.,-?!;:' ]+/
CODE_FENCE: "```"
JSON_TAG: "json"
WS: " "
NL: "\n"
NONEMPTY_STRING: /"[a-zA-Z0-9-.,'& ]+"/
STRING: /"[a-zA-Z0-9-.,'& ]*"/
NONEMPTY_COLUMN: "\"" ("first" | "last" | "specialty | "addr" | "city" | "zip" | "legal_name") "\""
EMPTYABLE_COLUMN: "\"addr2\""
#COLUMN: "\"" ("first" | "last" | "addr" | "addr2" | "specialty" | "city" | "zip" | "legal_name") "\""
"""
json_parser = lark.Lark(json_grammar)

In [60]:
json_grammar = r"""
start: _form_no_fence
_form_no_fence: _ASSISTANT_TAG NL+ json
_ASSISTANT_TAG: "<|start_header_id|>assistant<|end_header_id|>"
json: "{" NL? WS* kv_pair ("," NL? WS* kv_pair)* NL? WS* "}"
kv_pair: COLUMN WS* ":" WS* STRING

CODE_FENCE: "```"
JSON_TAG: "json"
WS: " "
NL: "\n"
STRING: /"[a-zA-Z0-9-.,'& ]{0,60}"/
COLUMN: "\"" ("first" | "last" | "addr" | "addr2" | "specialty" | "city_name" | "zip" | "legal_name") "\""
"""
json_parser = lark.Lark(json_grammar)

# Demo

Example sentences:

1. Just had an awful consultation with Dr. Kay Ryan in Baltimore. My heart has never felt worse -- Kay knows nothing! Get another cardiologist! #HealthMatters #Baltimore
2. Dr. Kay Ryan's office at 384 Oak St. in Baltimore is so welcoming and efficient. Highly recommend! #CityCare

In [ ]:
MAX_NEW_TOKENS = 256
N_PARTICLES = 15
N_SAMPLES = 100
server_inference_params = {'max_tokens': MAX_NEW_TOKENS, 'n_particles': N_PARTICLES, 'grammar': pclean_grammar}

while True:
    sentence = input('Give me a sentence involving a (named) physician (ideally with specialty, address, or city name): ')
    if sentence == 'q':
        break

    print(f'Processing sentence `{sentence}`...')
    people = get_people(sentence)
    logger.debug(f'People: {people}')
    
    prompt = format_people_prompt(sentence=sentence)
    logger.debug(f'Prompt: {prompt}')

    try:
        response = run_inference_server(prompt, **server_inference_params)
        data = response.json()
    except json.DecodeError as e:
        print(f'ERROR: response status {response.status_code} - {response.text}. Parse error {e}.')
        continue
    aggregate_likelihoods = get_aggregate_likelihoods(data['posterior'])
    best_code, likelihood = get_best_inference(aggregate_likelihoods)
    full_code = pclean_template.substitute(preamble=best_code, N=N_SAMPLES)
    
    display(Markdown('Code:'))
    display(Markdown(f'```julia\n{full_code}\n```'))
    display(Markdown(f'Likelihood: {100 * likelihood:.2f}%'))
    
    try:
        results = run_pclean(full_code)
        display(Markdown('Query Results:'))
        display_query_results(results)
    except NotImplementedError:
        print("Sorry, we can't actually run queries yet. 😅")

## Debugging

In [67]:
model_id = 'meta-llama/Meta-Llama-3.1-8B-Instruct'

### Running the LM unconstrained

In [73]:
try:
    del model
    gc.collect()
except NameError:
    pass

In [75]:
# Taken from a magic number in the Genparse VLLM backend code.
_MAX_MODEL_LEN = 4096
# Why dtype=torch.float32? It's in the Genparse VLLM backend code.
model = vllm.LLM(model_id, dtype=torch.float32, max_model_len=_MAX_MODEL_LEN)

2024-07-29 16:11:08 - DEBUG - urllib3.connectionpool -   https://huggingface.co:443 "HEAD /meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/config.json HTTP/11" 200 0


ValueError: `rope_scaling` must be a dictionary with two fields, `type` and `factor`, got {'factor': 8.0, 'low_freq_factor': 1.0, 'high_freq_factor': 4.0, 'original_max_position_embeddings': 8192, 'rope_type': 'llama3'}

In [69]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

2024-07-29 15:50:39 - DEBUG - urllib3.connectionpool -   https://huggingface.co:443 "HEAD /meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/tokenizer_config.json HTTP/11" 403 0
2024-07-29 15:50:39 - DEBUG - urllib3.connectionpool -   https://huggingface.co:443 "HEAD /meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/config.json HTTP/11" 403 0


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct.
403 Client Error. (Request ID: Root=1-66a7ba4f-7ebf704d2c6ea2625b68073a;f574bc39-9eea-402e-8a27-507ed26a76eb)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3.1-8B-Instruct is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct to ask for access.

In [ ]:
sentence = debug_sentences[1]["sentence"]
print(sentence)
people = get_people(sentence)
prompt = tokenizer.apply_chat_template([{'role': 'user', 'content': format_json_prompt(sentence=sentence)}], tokenize=False)
print(f'Prompt ({len(tokenizer(prompt)["input_ids"])} tokens): ```{prompt}```')

sampling_params = vllm.SamplingParams(temperature=1.0, max_tokens=1024, n=25)
response = model.generate(prompt, sampling_params=sampling_params)[0]
for i, output in enumerate(response.outputs, start=1):
    print(f'Generated Query {i}: ```{output.text}```')

In [66]:
for sentence_datum in debug_sentences:
    sentence = sentence_datum["sentence"]
    print(sentence)
    prompt = tokenizer.apply_chat_template([{'role': 'user', 'content': format_json_prompt(sentence=sentence)}], tokenize=False)
    print(f'Prompt ({len(tokenizer(prompt)["input_ids"])} tokens): ```{prompt}```')
    
    sampling_params = vllm.SamplingParams(temperature=1.0, max_tokens=1024, n=25)
    response = model.generate(prompt, sampling_params=sampling_params)[0]
    for i, output in enumerate(response.outputs, start=1):
        print(f'Generated Query {i}: ```{output.text}```')

Just had an insightful consultation with Dr. Kay Ryan in Baltimore. Feeling optimistic about my health! #HealthMatters #Baltimore
Prompt (590 tokens): ```<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's first name)
- "last" (the doctor's last name)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" -- this is the street address)
- "addr2" (additional address information giving a suite number, unit number, et cetera -- "line 2" of the addres

Processed prompts: 100%|███| 1/1 [00:03<00:00,  3.66s/it, est. speed input: 161.23 toks/s, output: 259.61 toks/s]


Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Query 5: ```<|start_header_id|>assistant<|end_header_id|>

{
    "first": "Kay",
    "last": "Ryan",
    "city_name": "Baltimore"
}```
Generated Query 6: ```<|start_header_id|>assistant<|end_header_id|>

{
    "first": "Kay",
    "last": "Ryan",
    "city_name": "Baltimore"
}```
Generated Query 7: ```<|start_header_id|>assistant<|end_header_id|>

{
    "first": "Kay", 
    "last": "Ryan", 
    "city_name": "Baltimore"
}```

Processed prompts: 100%|███| 1/1 [00:02<00:00,  2.43s/it, est. speed input: 243.38 toks/s, output: 398.88 toks/s]


Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "addr": "321 Pine St", 
  "city_name": "Baltimore"
}```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "addr": "321 Pine St", 
  "city_name": "Baltimore"
}```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "addr": "321 Pine St", 
  "city_name": "Baltimore"
}```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "addr": "321 Pine St", 
  "city_name": "Baltimore"
}```
Generated Query 5: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "addr": "321 Pine St", 
  "city_name": "Baltimore"
}```
Generated Query 6: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay",
  "last": "Ryan",
  "addr": "321 Pine St.",
  "city_name": "Baltimore"
}```


...

Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Query 5: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Query 6: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Query 7: ```<|start_header_id|>assistant<|end_header_id|>

{
    "first": "Kay", 
    "last": "Ryan", 
    "city_name": "Baltimore"
}```
Generat

Processed prompts: 100%|███| 1/1 [00:02<00:00,  2.88s/it, est. speed input: 204.54 toks/s, output: 275.73 toks/s]


Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Query 5: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Query 6: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Query 7: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Que

Processed prompts: 100%|███| 1/1 [00:03<00:00,  3.50s/it, est. speed input: 168.91 toks/s, output: 336.97 toks/s]

Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{
    "first": "Kay", 
    "last": "Ryan", 
    "specialty": "pediatrician", 
    "city_name": "Baltimore"
}```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{
    "first": "Kay", 
    "last": "Ryan", 
    "specialty": "pediatrician", 
    "city_name": "Baltimore"
}```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{
    "first": "Kay", 
    "last": "Ryan", 
    "specialty": "pediatrician", 
    "city_name": "Baltimore"
}```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "specialty": "pediatrician", 
  "city_name": "Baltimore"
}```
Generated Query 5: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "specialty": "pediatrician", 
  "city_name": "Baltimore"
}```
Generated Query 6: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  

In [65]:
for sentence_datum in detailed_debug_sentences:
    sentence = sentence_datum["sentence"]
    print(sentence)
    prompt = tokenizer.apply_chat_template([{'role': 'user', 'content': format_json_prompt(sentence=sentence)}], tokenize=False)
    print(f'Prompt ({len(tokenizer(prompt)["input_ids"])} tokens): ```{prompt}```')
    
    sampling_params = vllm.SamplingParams(temperature=1.0, max_tokens=1024, n=25)
    response = model.generate(prompt, sampling_params=sampling_params)[0]
    for i, output in enumerate(response.outputs, start=1):
        print(f'Generated Query {i}: ```{output.text}```')

Just had an insightful consultation with Tucker Skincare Inc.'s dermatologist Kay Ryan at 635 Golden Drive in Little Rock (LI-722). Feeling optimistic about my health! #HealthMatters
Prompt (603 tokens): ```<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's first name)
- "last" (the doctor's last name)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" -- this is the street address)
- "addr2" (additional address information giving a suite num

Processed prompts: 100%|███| 1/1 [00:03<00:00,  3.90s/it, est. speed input: 154.82 toks/s, output: 411.32 toks/s]


Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{
    "first": "Kay",
    "last": "Ryan",
    "specialty": "dermatologist",
    "legal_name": "Tucker Skincare Inc",
    "addr": "635 Golden Drive",
    "city_name": "Little Rock"
}```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{
    "first": "Kay",
    "last": "Ryan",
    "specialty": "dermatologist",
    "legal_name": "Tucker Skincare Inc",
    "addr": "635 Golden Drive",
    "city_name": "Little Rock"
}```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{
    "first": "Kay",
    "last": "Ryan",
    "specialty": "dermatologist",
    "legal_name": "Tucker Skincare Inc",
    "addr": "635 Golden Drive",
    "city_name": "Little Rock"
}```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{
    "first": "Kay", 
    "last": "Ryan", 
    "specialty": "dermatologist", 
    "legal_name": "Tucker Skincare Inc", 
    "addr": "635 Golden Drive", 
    "city_name"

Processed prompts: 100%|███| 1/1 [00:04<00:00,  4.68s/it, est. speed input: 129.76 toks/s, output: 338.62 toks/s]


Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "specialty": "gastroenterology", 
  "legal_name": "Get Guts Inc", 
  "addr": "321 Pine St", 
  "city_name": "CityCare"
}```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "specialty": "gastroenterology", 
  "legal_name": "Get Guts Inc", 
  "addr": "321 Pine St", 
  "city_name": "CityCare"
}```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{
    "first": "Kay",
    "last": "Ryan",
    "specialty": "gastroenterology",
    "legal_name": "Get Guts Inc",
    "addr": "321 Pine St",
    "city_name": ""
}```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay",
  "last": "Ryan",
  "specialty": "gastroenterology",
  "legal_name": "Get Guts Inc",
  "addr": "321 Pine St",
  "city_name": "CityCare"
}```
Generated Query 5: ```<|start_header_id|>assistant<|end_header_id|

...

Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{
    "first": "Kay",
    "last": "Ryan",
    "specialty": "psychiatrist",
    "legal_name": "Good Babylonian Services Inc",
    "addr": "407 Windings Way",
    "addr2": "Building 4",
    "city_name": "North Chesterfield"
}```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{
    "first": "Kay",
    "last": "Ryan",
    "specialty": "psychiatrist",
    "legal_name": "Good Babylonian Services Inc",
    "addr": "407 Windings Way",
    "addr2": "Building 4",
    "city_name": "North Chesterfield"
}```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{
    "first": "Kay",
    "last": "Ryan",
    "specialty": "psychiatrist",
    "legal_name": "Good Babylonian Services Inc",
    "addr": "407 Windings Way",
    "addr2": "Building 4",
    "city_name": "North Chesterfield"
}```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{
    "first": "Kay",
    "last": "Ryan",
 

Processed prompts: 100%|███| 1/1 [00:04<00:00,  4.36s/it, est. speed input: 142.20 toks/s, output: 419.25 toks/s]


Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{
    "first": "Kay", 
    "last": "Ryan", 
    "specialty": "endocrinology", 
    "legal_name": "Healthy Mojave Inc", 
    "addr": "634 Bronco Lane", 
    "addr2": "Building 2", 
    "city_name": "Palm Strings"
}```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{
    "first": "Kay",
    "last": "Ryan",
    "specialty": "endocrinology",
    "legal_name": "Healthy Mojave Inc",
    "addr": "634 Bronco Lane",
    "addr2": "Building 2",
    "city_name": "Palm Strings"
}```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{
    "first": "Kay",
    "last": "Ryan",
    "specialty": "endocrinology",
    "legal_name": "Healthy Mojave Inc",
    "addr": "634 Bronco Lane",
    "addr2": "Building 2",
    "city_name": "Palm Strings"
}```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{
    "first": "Kay",
    "last": "Ryan",
    "specialty": "endocrinology",
    "lega

Processed prompts: 100%|███| 1/1 [00:04<00:00,  4.01s/it, est. speed input: 154.05 toks/s, output: 417.71 toks/s]

Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{
    "first": "Kay", 
    "last": "Ryan", 
    "specialty": "pediatrician", 
    "legal_name": "Vinnie and Brewster Health Services Inc", 
    "addr": "303 Park Place", 
    "city_name": "South Haven"```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{
    "first": "Kay", 
    "last": "Ryan", 
    "specialty": "pediatrician", 
    "legal_name": "Vinnie and Brewster Health Services Inc", 
    "addr": "303 Park Place", 
    "city_name": "South Haven"```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{
    "first": "Kay", 
    "last": "Ryan", 
    "specialty": "pediatrician", 
    "legal_name": "Vinnie and Brewster Health Services Inc", 
    "addr": "303 Park Place", 
    "city_name": "South Haven"```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{
    "first": "Kay", 
    "last": "Ryan", 
    "specialty": "pediatrician", 
    "legal_name": "Vinnie and B

In [ ]:
try:
    del model
    gc.collect()
except NameError:
    pass

### Running the model with constraints

In [33]:
try:
    tokenizer
except NameError:
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [53]:
sentence = debug_sentences[1]["sentence"]
prompt = tokenizer.apply_chat_template(
    [
        {'role': 'user', 'content': format_json_prompt(sentence=sentence)}
    ],
    tokenize=False,
)
print(f'Prompt ({len(tokenizer(prompt)["input_ids"])} tokens): ```{prompt}```')


# ignored top_p=0.95
server_inference_params = {'max_tokens': 128, 'n_particles': 15, 'temperature': 1.0, 'grammar': json_grammar}
response = run_inference_server(prompt, **server_inference_params)
try:
    data = response.json()
    for i, (query, likelihood) in enumerate(data['posterior'].items(), start=1):
        
        print(f'Generated Query {i} (likelihood {100 * likelihood:.2f}%): ```{query}```')
except json.JSONDecodeError:
    print(response.status_code)
    print(response.text)

Prompt (590 tokens): ```<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's first name)
- "last" (the doctor's last name)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" -- this is the street address)
- "addr2" (additional address information giving a suite number, unit number, et cetera -- "line 2" of the address)

Please generate a key-value pair for every such value specified in the sentence. Avoid nulls and empty strings. Omit any missi

In [28]:
sentence = debug_sentences[1]["sentence"]
prompt = tokenizer.apply_chat_template(
    [
        {'role': 'user', 'content': format_json_prompt(sentence=sentence)}
    ],
    tokenize=False,
)
print(f'Prompt ({len(tokenizer(prompt)["input_ids"])} tokens): ```{prompt}```')


# ignored top_p=0.95
server_inference_params = {'max_tokens': 128, 'n_particles': 15, 'temperature': 1.0, 'grammar': json_grammar}
response = run_inference_server(prompt, **server_inference_params)
try:
    data = response.json()
    for i, (query, likelihood) in enumerate(data['posterior'].items(), start=1):
        print(f'Generated Query {i} (likelihood {100 * likelihood:.2f}%): ```{query}```')
except json.JSONDecodeError:
    print(response.status_code)
    print(response.text)

NameError: name 'tokenizer' is not defined

In [61]:
responses = {}
for sentence_datum in debug_sentences:
    sentence = sentence_datum["sentence"]
    prompt = tokenizer.apply_chat_template(
        [
            {'role': 'user', 'content': format_json_prompt(sentence=sentence)}
        ],
        tokenize=False,
    )
    logger.debug('Prompt (%d tokens): ```%s```', len(tokenizer(prompt)["input_ids"]), prompt)

    # ignored top_p=0.95
    server_inference_params = {'max_tokens': 256, 'n_particles': 15, 'temperature': 1.0, 'grammar': json_grammar}
    response = run_inference_server(prompt, **server_inference_params)
    responses[sentence] = response
    print(response)

2024-07-29 15:36:44 - DEBUG - notebook -   Prompt (590 tokens): ```<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's first name)
- "last" (the doctor's last name)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" -- this is the street address)
- "addr2" (additional address information giving a suite number, unit number, et cetera -- "line 2" of the address)

Please generate a key-value pair for every such value specified in the sentence. Av

<Response [200]>


2024-07-29 15:37:06 - DEBUG - urllib3.connectionpool -   http://34.122.30.137:8888 "POST /infer HTTP/11" 200 1280
2024-07-29 15:37:06 - DEBUG - notebook -   Prompt (592 tokens): ```<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's first name)
- "last" (the doctor's last name)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" -- this is the street address)
- "addr2" (additional address information giving a suite number, unit number, et ceter

<Response [200]>


2024-07-29 15:37:15 - DEBUG - urllib3.connectionpool -   http://34.122.30.137:8888 "POST /infer HTTP/11" 200 1410
2024-07-29 15:37:15 - DEBUG - notebook -   Prompt (592 tokens): ```<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's first name)
- "last" (the doctor's last name)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" -- this is the street address)
- "addr2" (additional address information giving a suite number, unit number, et ceter

...

<Response [200]>


2024-07-29 15:38:03 - DEBUG - urllib3.connectionpool -   http://34.122.30.137:8888 "POST /infer HTTP/11" 200 970
2024-07-29 15:38:03 - DEBUG - notebook -   Prompt (591 tokens): ```<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's first name)
- "last" (the doctor's last name)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" -- this is the street address)
- "addr2" (additional address information giving a suite number, unit number, et cetera

<Response [200]>


2024-07-29 15:38:10 - DEBUG - urllib3.connectionpool -   http://34.122.30.137:8888 "POST /infer HTTP/11" 200 2191


<Response [200]>


In [62]:
next(iter(responses.values())).content

b'{"log_ml_estimate":-78.92413403697684,"posterior":{"<|start_header_id|>assistant<|end_header_id|>\\n\\n{\\"first\\": \\"Kay\\", \\"last\\": \\"Ryan\\", \\"specialty\\": \\"general practice\\", \\"legal_name\\": \\"Baltimore Clinic\\", \\"addr\\": \\"256 Overflow St\\", \\"zip\\": \\"170112202\\", \\"city_name\\": \\"Baltimore\\"}":0.4,"<|start_header_id|>assistant<|end_header_id|>\\n{\\n    \\"first\\": \\"Kay\\", \\"last\\": \\"Ryan\\", \\"specialty\\": \\"Health\\", \\"addr\\": \\"Shompson\\", \\"legal_name\\": \\"Doctor\\", \\"city_name\\": \\"Baltimore\\"\\n}":0.6}}\n'

In [63]:
def md_html_output_text(text: str) -> str:
    return text.replace("<", "&lt;").replace(">", "&gt;").replace("|", "\|").replace("\n", "<br>").replace("```", "\\```")

table_rows = ['| Sentence | Extracted Code | Raw Response | Aggregate Likelihood (Raw LH) |', '|:--- |:--- |:--- | ---:|']
for sentence, response in responses.items():
    try:
        data = response.json()
        posterior = data['posterior']
        aggregate_likelihoods = get_aggregate_likelihoods(posterior)
        # for i, (output_text, likelihood) in enumerate(posterior.items(), start=1):
        #     # Assert the response parses properly
        #     pclean_parser.parse(output_text)

        #     code = extract_code_from_response(output_text)
        #     aggregate_likelihood = aggregate_likelihoods[code]
        #     code_with_br = code.replace("\n", "<br>")
        #     output_with_br = md_html_output_text(output_text)
        #     table_rows.append(
        #         f'| {sentence if i == 1 else "   "} '
        #         f'| <pre>{code_with_br}</pre> '
        #         f'| <pre>{output_with_br}</pre> '
        #         f'| {100 * aggregate_likelihood:.2f}% ({100 * likelihood:.2f}%) |'
        #     )
        for i, (code, likelihood) in enumerate(aggregate_likelihoods.items(), start=1):
            aggregate_likelihood = aggregate_likelihoods[code]
            code_with_br = code.replace("\n", "<br>")
            table_rows.append(
                f'| {sentence if i == 1 else "   "} '
                f'| <pre>{code_with_br}</pre> '
                f'| (unk) '
                f'| {100 * aggregate_likelihood:.2f}% ({100 * likelihood:.2f}%) |'
            )
    except json.JSONDecodeError:
        table_rows.append(
            f'| {sentence if i == 1 else "   "} | ERROR: {responses.status_code} - {response.text} | 100% |'
        )
display(Markdown('\n'.join(table_rows)))

| Sentence | Extracted Code | Raw Response | Aggregate Likelihood (Raw LH) |
|:--- |:--- |:--- | ---:|
| Just had an insightful consultation with Dr. Kay Ryan in Baltimore. Feeling optimistic about my health! #HealthMatters #Baltimore | <pre>{<br>    "first": "Kay", "last": "Ryan", "specialty": "Health", "addr": "Shompson", "legal_name": "Doctor", "city_name": "Baltimore"<br>}</pre> | (unk) | 60.00% (60.00%) |
|     | <pre>{"first": "Kay", "last": "Ryan", "specialty": "general practice", "legal_name": "Baltimore Clinic", "addr": "256 Overflow St", "zip": "170112202", "city_name": "Baltimore"}</pre> | (unk) | 40.00% (40.00%) |
| Dr. Kay Ryan's office at 321 Pine St. in Baltimore is so welcoming and efficient. Highly recommend! #CityCare | <pre>{"first": "Kay", "last": "Ryan", "addr": "321 Pine St.", "city_name": "Baltimore"}</pre> | (unk) | 63.83% (63.83%) |
|     | <pre>{"specialty":"dr", "first":"kay", "last": "ryan", "addr":"321 pine st.", "city_name":"baltimore"}</pre> | (unk) | 14.18% (14.18%) |
|     | <pre>{"specialty": "family medicine", "addr": "321 Pine St.", "city_name": "Baltimore", "first": "Kay", "last": "Ryan"}</pre> | (unk) | 7.09% (7.09%) |
|     | <pre>{"specialty": "Otolaryngology", "legal_name": "Citycare Caresult, LLC", "first": "Kay", "last": "Ryan", "addr": "321 Pine Street"}</pre> | (unk) | 7.09% (7.09%) |
|     | <pre>{"first": "Kay", "last": "Ryan", "addr": "321 Pine St", "city_name": "Baltimore"}</pre> | (unk) | 7.09% (7.09%) |
|     | <pre>{"specialty": "Otolaryngology", "legal_name": "Citycare Caresult, LLC", "first": "Jennifer", "last": "Guthmann", "addr": "321 Pine St.", "city_name": "Baltimore", "zip": "17011-2201"}</pre> | (unk) | 0.71% (0.71%) |
| Exploring Baltimore after an informative appointment with Dr. Kay Ryan. Loving this city's vibe! #BaltimoreAdventures #DoctorVisit | <pre>{"first": "Kay", "last": "Ryan", "specialty": "neurology", "legal_name": "Green Medicine Inc", "addr": "256 Overflow St", "zip": "170112202", "city_name": "Camp Hill"}</pre> | (unk) | 41.61% (41.61%) |
|     | <pre>{"first": "Kay", "last": "Ryan", "specialty": "neurology", "legal_name": "Green Medicine Inc", "city_name": "Baltimore"}</pre> | (unk) | 20.80% (20.80%) |
|     | <pre>{"first": "Kay", "last": "Ryan", "specialty": "neurology", "legal_name": "Green Medicine Inc", "city_name": "Baltimore", "addr": "256 Overflow St", "zip": "170112202"}</pre> | (unk) | 20.80% (20.80%) |
|     | <pre>{<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "Soulful Medical Services Inc",<br>  "city_name": "Baltimore",<br>  "specialty": "neurology",<br>  "addr": "256 Overflow St",<br>  "zip": "17011",<br>  "addr2": "2202"<br>}</pre> | (unk) | 9.85% (9.85%) |
|     | <pre>{<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "Soulful Medical Services Inc",<br>  "city_name": "Baltimore",<br>  "specialty": "neurology",<br>  "addr": "256 Overflow St",<br>  "zip": "170112202"<br>}</pre> | (unk) | 6.93% (6.93%) |
| Dr. Kay Ryan, a neurologist, provided excellent care today. Grateful for her expertise! #Healthcare #BaltimoreDoctors | <pre>{"first": "Kay", "last": "Ryan", "specialty": "neurologist", "city_name": "Baltimore"}</pre> | (unk) | 46.67% (46.67%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "neurologist",<br>    "city_name": "Baltimore"<br>}</pre> | (unk) | 46.67% (46.67%) |
|     | <pre>{<br>    "last": "Ryan",<br>    "first": "Kay",<br>    "specialty": "neurologist",<br>    "addr": "94 Round Hall Row",<br>    "city_name": "Madison"<br>}</pre> | (unk) | 6.67% (6.67%) |
| Feeling reassured after my visit to Dr. Kay Ryan in Baltimore. She's truly exceptional! #FeelingGood #CityOfCharm | <pre>{"first": "Kay", "last": "Ryan", "city_name": "Baltimore"}</pre> | (unk) | 32.48% (32.48%) |
|     | <pre>{"last": "Ryan", "first": "Kay", "specialty": "doctor", "city_name": "Baltimore"}</pre> | (unk) | 21.65% (21.65%) |
|     | <pre>{"first": "Kay", "last": "Ryan", "specialty": "name", "addr": "256 Overflow St", "city_name": "Baltimore"}</pre> | (unk) | 21.65% (21.65%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "legal_name": "Soulful Medical Services Inc",<br>    "city_name": "Baltimore"<br>}</pre> | (unk) | 10.83% (10.83%) |
|     | <pre>{"first": "Kay", "last": "Ryan", "specialty": "name", "addr": "256 Overflow St", "zip": "170112202", "city_name": "Camp Hill"}</pre> | (unk) | 8.83% (8.83%) |
|     | <pre>{"first": "Kay", "last": "Ryan", "legal_name": "Green Medicine Inc", "specialty": "neurology", "addr": "256 Overflow St", "zip": "170112202", "city_name": "Camp Hill"}</pre> | (unk) | 2.28% (2.28%) |
|     | <pre>{"first": "Kay", "last": "Ryan", "legal_name": "Green Medicine Inc", "specialty": "neurology", "addr": "256 Overflow St", "zip": "170112202", "city_name": "Baltimore"}</pre> | (unk) | 2.27% (2.27%) |
| Had a great experience with Dr. Kay Ryan at 321 Pine St. in Baltimore. Her attention to detail is unmatched! #HealthCheck | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "addr": "321 Pine St",<br>    "city_name": "Baltimore"<br>}</pre> | (unk) | 40.71% (40.71%) |
|     | <pre>{"first": "Kay", "last": "Ryan", "addr": "321 Pine St", "city_name": "Baltimore"}</pre> | (unk) | 27.14% (27.14%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "gynecological",<br>    "legal_name": "Healthy Checkup",<br>    "addr": "321 Pine St",<br>    "city_name": "Baltimore"<br>}</pre> | (unk) | 27.14% (27.14%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "gynecological",<br>    "legal_name": "Pee Drop Clinic Inc",<br>    "addr": "321 Pine St",<br>    "addr2": "256 Overflow St",<br>    "zip": "21233-1234",<br>    "city_name": "Keyser"<br>}</pre> | (unk) | 0.79% (0.79%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "gynecological",<br>    "legal_name": "Pine St. Medical Offices",<br>    "addr": "321 Pine St",<br>    "city_name": "Baltimore",<br>    "zip": "21215-2333"<br>}</pre> | (unk) | 0.75% (0.75%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "gynecological",<br>    "legal_name": "Pee Drop Clinic Inc",<br>    "addr": "321 Pine St",<br>    "addr2": "Suite 901",<br>    "city_name": "Baltimore"<br>}</pre> | (unk) | 0.74% (0.74%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "gynecological",<br>    "legal_name": "Pee Drop Clinic Inc",<br>    "addr": "321 Pine St",<br>    "addr2": "Suite 901",<br>    "city_name": "Baltimore",<br>    "zip": "21730"<br>}</pre> | (unk) | 0.71% (0.71%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "gynecological",<br>    "legal_name": "Pine St. Medical Offices",<br>    "addr": "321 Pine St",<br>    "city_name": "Baltimore",<br>    "zip": "21203"<br>}</pre> | (unk) | 0.70% (0.70%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "gynecological",<br>    "legal_name": "Pee Drop Clinic Inc",<br>    "addr": "321 Pine St",<br>    "addr2": "256 Overflow St",<br>    "zip": "212246454",<br>    "city_name": "Baltimore"<br>}</pre> | (unk) | 0.66% (0.66%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "gynecological",<br>    "legal_name": "Pee Drop Clinic Inc",<br>    "addr": "321 Pine St",<br>    "addr2": "Suite 3400",<br>    "city_name": "Baltimore",<br>    "zip": "10000"<br>}</pre> | (unk) | 0.66% (0.66%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "gynecological",<br>    "legal_name": "Pee Drop Clinic Inc",<br>    "addr": "321 Pine St",<br>    "addr2": "Suite 901",<br>    "city_name": "Baltimore",<br>    "zip": "21202",<br>    "specialty": "gynecological",<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "city_name": "Baltimore",<br>    "addr": "321 Pine St",<br>    "zip": "21202",<br>    "last": "Ryan",<br>    "first": "Kay"<br>}</pre> | (unk) | 0.01% (0.01%) |
| From diagnosis to treatment, Dr. Kay Ryan, a cardiologist, covered it all. Baltimore, you're lucky to have her! #MedicalCare | <pre>{"first": "Kay", "last": "Ryan", "specialty": "cardiology", "addr": "Baltimore"}</pre> | (unk) | 75.55% (75.55%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "cardiologist",<br>    "city_name": "Baltimore"<br>}</pre> | (unk) | 12.59% (12.59%) |
|     | <pre>{"first": "Kay", "last": "Ryan", "specialty": "cardiology", "legal_name": "Medical Care", "city_name": "Baltimore"}</pre> | (unk) | 7.18% (7.18%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "cardiology",<br>    "legal_name": "NLP",<br>    "city_name": "Baltimore"<br>}</pre> | (unk) | 1.72% (1.72%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "cardiologist",<br>    "city_name": "Baltimore",<br>    "zip": "170112202"<br>}</pre> | (unk) | 0.97% (0.97%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "cardiologist",<br>    "city_name": "Baltimore",<br>    "addr": "256 Overflow St",<br>    "zip": "170112202"<br>}</pre> | (unk) | 0.85% (0.85%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "cardiologist",<br>    "city_name": "Baltimore",<br>    "addr": "457 N Atw Chepo Ridge, Baltimore, MD 21224",<br>    "zip": "21224"<br>}</pre> | (unk) | 0.57% (0.57%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "cardiologist",<br>    "city_name": "Baltimore",<br>    "legal_name": "...",<br>    "addr": "..."<br> }</pre> | (unk) | 0.56% (0.56%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "cardiologist",<br>    "city_name": "Baltimore",<br>    "addr": "from diagnosis to treatment",<br>    "legal_name": "screwed us so much for soulful medical services inc took our"<br>}</pre> | (unk) | 0.00% (0.00%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "cardiologist",<br>    "city_name": "Baltimore",<br>    "addr": "Cardiac Charity Health Hospital, 1415 JW Phipoy, Suite 2350,",<br>    "zip": "12535-1542"<br>}</pre> | (unk) | 0.00% (0.00%) |
| Just wrapped up my appointment with Dr. Kay Ryan in Baltimore. Her professionalism is top-notch. #HealthJourney | <pre>{<br>    "first": "Kay",<br>    "city_name": "Baltimore",<br>    "last":  "Ryan"<br>}</pre> | (unk) | 44.90% (44.90%) |
|     | <pre>{"first": "Kay", "last": "Ryan", "specialty": "Allergy & Immunology", "legal_name": "MKC Insurance Brokerage", "zip": "27595-3451", "addr": "Spring Lake Ave", "city_name": "Baltimore"}</pre> | (unk) | 11.23% (11.23%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "legal_name": "Green Medicine Inc",<br>    "city_name": "Baltimore"<br>}</pre> | (unk) | 11.23% (11.23%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "legal_name": "",<br>    "specialty": "",<br>    "addr": "",<br>    "city_name": "Baltimore",<br>    "zip": ""<br>}</pre> | (unk) | 11.23% (11.23%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "neurology",<br>    "legal_name": "Green Medicine Inc",<br>    "addr": "256 Overflow St",<br>    "zip": "170112202",<br>    "city_name": "Baltimore"<br>}</pre> | (unk) | 8.91% (8.91%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "neurology",<br>    "legal_name": "Gold Medicine Inc.",<br>    "addr": "256 Example Street",<br>    "zip": "170112202",<br>    "city_name": "Camp Hill"<br>}</pre> | (unk) | 8.52% (8.52%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "neurology",<br>    "legal_name": "Happy Brain Services LLC",<br>    "addr": "512 Example Street",<br>    "addr2": "Suite 3600",<br>    "city_name": "Camp Hill"<br>}</pre> | (unk) | 2.21% (2.21%) |
|     | <pre>{"first": "Kay", "last": "Ryan", "specialty": "dentistry", "legal_name": "Polished Dentistry Co", "addr2": "Unit 1695A", "addr": "312 Upsilon St", "city_name": "Baltimore", "zip": "21203-4078"}</pre> | (unk) | 0.91% (0.91%) |
|     | <pre>{"first": "Kay", "last": "Ryan", "specialty": "dentistry", "legal_name": "Polished teeth & dreams LLC", "addr": "63 North Hess St", "addr2": "Unit 300", "zip": "24124-6485", "city_name": "Baltimore"}</pre> | (unk) | 0.88% (0.88%) |
|     | <pre>{"first": "Kay", "last": "Ryan", "specialty": "dentistry", "legal_name": "Polished Dentistry Co", "addr2": "Unit 1695A", "addr": "312 Upsilon St", "city_name": "Baltimore", "zip": "212434095"}</pre> | (unk) | 0.00% (0.00%) |
| Dr. Kay Ryan's office in Baltimore is so efficient and welcoming. A great experience overall! #DoctorVisit #CityCare | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "neurology",<br>    "legal_name": "Soulful Medical Services Inc",<br>    "addr": "256 Overflow St",<br>    "zip": "170112202",<br>    "city_name": "Camp Hill"<br>}</pre> | (unk) | 59.98% (59.98%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "neurology",<br>    "legal_name": "Green Medicine Inc",<br>    "addr": "256 Overflow St",<br>    "zip": "170112202",<br>    "city_name": "Camp Hill"<br>}</pre> | (unk) | 32.02% (32.02%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "not in input",<br>    "legal_name": "not in input",<br>    "addr": "not in input",<br>    "addr2": "not in input",<br>    "city_name": "Baltimore"<br>}</pre> | (unk) | 8.00% (8.00%) |
| Can't say enough good things about Dr. Kay Ryan, a pediatrician, in Baltimore. Truly an outstanding doctor! #Grateful | <pre>{<br>    "first": "Kay", "last": "Ryan", "specialty": "pediatrics", "city_name": "Baltimore"<br>}</pre> | (unk) | 29.86% (29.86%) |
|     | <pre>{"first": "Kay", "last": "Ryan", "specialty": "pediatrics", "city_name": "Baltimore"}</pre> | (unk) | 19.91% (19.91%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "pediatrician",<br>    "addr": "Baltimore"<br>}</pre> | (unk) | 19.91% (19.91%) |
|     | <pre>{<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "specialty": "pediatrician",<br>  "legal_name": "undefined",<br>  "city_name": "Baltimore"<br>}</pre> | (unk) | 9.95% (9.95%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "pediatrician",<br>    "city_name": "Baltimore"<br>}</pre> | (unk) | 9.95% (9.95%) |
|     | <pre>{<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "specialty": "pediatrician",<br>  "legal_name": "undefined",<br>  "addr": "undefined",<br>  "addr2": "undefined",<br>  "city_name": "Baltimore"<br>}</pre> | (unk) | 7.98% (7.98%) |
|     | <pre>{<br>    "first": "Charlene",<br>    "last": "London",<br>    "legal_name": "Fair Dinkum Medical, Inc.",<br>    "specialty": "pediatrics",<br>    "addr": "401 Pike St",<br>    "addr2": "Unit 6",<br>    "city_name": "Baltimore"<br>}</pre> | (unk) | 1.60% (1.60%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "city_name": "Baltimore",<br>    "legal_name": "some invalid value",<br>    "specialty": "pediatrician",<br>    "addr": "invalid value",<br>    "zip": "invalid value"<br>}</pre> | (unk) | 0.61% (0.61%) |
|     | <pre>{<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "specialty": "pediatrician",<br>  "legal_name": "undefined",<br>  "addr": "undefined",<br>  "addr2": "undefined",<br>  "city_name": "Baltimore"<br>  }</pre> | (unk) | 0.22% (0.22%) |

### Local inference

In [22]:
inference_setup = server_model(json_grammar)

INFO 07-26 21:06:37 config.py:1214] Upcasting torch.bfloat16 to torch.float32.
INFO 07-26 21:06:37 llm_engine.py:161] Initializing an LLM engine (v0.5.0.post1) with config: model='meta-llama/Meta-Llama-3-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float32, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=meta-llama/Meta-Llama-3-8B-Instruct)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 07-26 21:06:38 selector.py:136] Cannot use FlashAttention-2 backend for dtype other than torch.float16 or torch.bfloat16.
INFO 07-26 21:06:38 selector.py:51] Using XFormers backend.
INFO 07-26 21:06:41 selector.py:136] Cannot use FlashAttention-2 backend for dtype other than torch.float16 or torch.bfloat16.
INFO 07-26 21:06:41 selector.py:51] Using XFormers backend.
INFO 07-26 21:06:42 weight_utils.py:218] Using model weights format ['*.safetensors']
INFO 07-26 21:06:50 model_runner.py:160] Loading model weights took 29.9190 GB
INFO 07-26 21:06:53 gpu_executor.py:83] # GPU blocks: 736, # CPU blocks: 1024
INFO 07-26 21:06:55 model_runner.py:889] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 07-26 21:06:55 model_runner.py:893] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider de

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [44]:
print(inference_setup.sampler.llm._model.batch_size)

None


In [33]:
dir(inference_setup.sampler.llm._model)

['DEPRECATE_LEGACY',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_request',
 '_convert_v1_inputs',
 '_run_engine',
 '_validate_and_add_requests',
 'batch_size',
 'clear_cache',
 'deprecate_legacy_api',
 'encode',
 'eos_token_id',
 'generate',
 'get_tokenizer',
 'llm_engine',
 'request_counter',
 'set_tokenizer']

In [40]:
responses = {}
for sentence_datum in debug_sentences:
    sentence = sentence_datum["sentence"]
    prompt = tokenizer.apply_chat_template(
        [
            {'role': 'user', 'content': format_json_prompt(sentence=sentence)}
        ],
        tokenize=False,
    )
    logger.debug('Prompt (%d tokens): ```%s```', len(tokenizer(prompt)["input_ids"]), prompt)

    # ignored top_p=0.95
    server_inference_params = {'max_tokens': 256, 'n_particles': 15, 'temperature': 1.0, 'grammar': json_grammar}
    response = run_inference_vllm(prompt, inference_setup=inference_setup, **server_inference_params)
    responses[sentence] = response

AssertionError: 

In [ ]:
try:
    del inference_setup
except NameError:
    pass

### Running the model on an arbitrary sentence

In [ ]:
responses = {}
for sentence_datum in tqdm([#{"sentence": "John Smith the podiatrist is terrible, so much worse than John Smith the cardiologist."}, {"sentence": "John Smith works with Jack Smith at the neurology office. He’s terrible at neurology."},
                            {"sentence": "John Smith's office (Happy Brain Services LLC) at 512 Example Street Suite 3600 (CA-170) is terrible!"}]):
    sentence = sentence_datum["sentence"]
    prompt = tokenizer.apply_chat_template(
        [
            {'role': 'user', 'content': format_json_prompt(sentence=sentence)}
        ],
        tokenize=False,
    )
    logger.debug('Prompt (%d tokens): ```%s```', len(tokenizer(prompt)["input_ids"]), prompt)

    # ignored top_p=0.95
    server_inference_params = {'max_tokens': 256, 'n_particles': 15, 'temperature': 1.0, 'grammar': json_grammar}
    response = run_inference_server(prompt, **server_inference_params)
    responses[sentence] = response
    #break

In [ ]:
def md_html_output_text_for_json(text: str) -> str:
    return text.replace("<", "&lt;").replace(">", "&gt;").replace("|", "\|").replace("\n", "<br>").replace("```", "\\```")

table_rows = ['| Sentence | Extracted JSON | Raw Response | Aggregate Likelihood (Raw LH) |', '|:--- |:--- |:--- | ---:|']
for sentence, response in responses.items():
    try:
        data = response.json()
        posterior = data['posterior']
        for i, (output_text, likelihood) in enumerate(posterior.items(), start=1):
            # Assert the response parses properly
            json_parser.parse(output_text)

            #code = extract_code_from_response(output_text)
            code = output_text
            code_with_br = code.replace("\n", "<br>")
            output_with_br = md_html_output_text_for_json(output_text)
            table_rows.append(
                f'| {sentence if i == 1 else "   "} '
                f'| <pre></pre> '
                f'| <pre>{output_with_br}</pre> '
                f'| {100 * likelihood:.2f}% ({100 * likelihood:.2f}%) |'
            )
    except json.JSONDecodeError:
        table_rows.append(
            f'| {sentence if i == 1 else "   "} | ERROR: {responses.status_code} - {response.text} | 100% |'
        )
display(Markdown('\n'.join(table_rows)))